# 'subway_feature.csv'로 '역세권', '구_역세권_랭크' 피처 생성하기
- 역세권: <br> 500m 이내에 지하철역이 있을 경우 역세권 | 아닐 경우 비역세권 <br>
역세권이라 하더라도, 역세권 자체보다도 '구'가 아파트 가격에 더 큰 영향을 미치기 때문에 최초에 역세권 피처를 생성할 때는 '구'명까지 붙임 <br>
이후 다중공산성을 완화하기 위해 구명은 제외
- 구_역세권_랭크: 구별 역세권/비역세권의 target의 평균을 구해 총 50위의 랭크를 매김 (25 * 2)
#### 1. 원본 데이터에서 NA인 X, Y좌표 보완
#### 2. subway_feature 에 '구' 단위 추가
#### 3. 같은 '구'에 속한 아파트와 지하철역 거리 계산 후 역세권/비역세권 산정
#### 4. 각 구 역세권/비역세권별 target 평균으로 랭크 산정

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

## NAVER API KEY 불러오기
NAVER_API_KEY_ID = os.getenv("NAVER_API_KEY_ID")
NAVER_API_KEY = os.getenv("NAVER_API_KEY")

## csv file path
PATH_SUBWAY = os.getenv("PATH_SUBWAY")
PATH_TRAIN = os.getenv("PATH_TRAIN")
PATH_TEST = os.getenv("PATH_TEST")
PATH_RECENT = os.getenv("PATH_RECENT")

In [2]:
# utils
import pandas as pd
import numpy as np
from tqdm import tqdm
import warnings;warnings.filterwarnings('ignore')

In [3]:
df_train = pd.read_csv(PATH_TRAIN)
df_test = pd.read_csv(PATH_TEST)

In [4]:
# train/test 구분을 위한 칼럼을 하나 만들어 줍니다.
df_train['is_test'] = 0
df_test['is_test'] = 1

# 하나의 데이터프레임으로 만들어줍니다.
concat = pd.concat([df_train, df_test])

In [5]:
concat.isna().sum()

시군구                             0
번지                            227
본번                             75
부번                             75
아파트명                         2136
전용면적(㎡)                         0
계약년월                            0
계약일                             0
층                               0
건축년도                            0
도로명                             0
해제사유발생일                   1121899
등기신청일자                          0
거래유형                            0
중개사소재지                          0
k-단지분류(아파트,주상복합등등)         877273
k-전화번호                     876850
k-팩스번호                     879348
단지소개기존clob                1058958
k-세대타입(분양형태)               876125
k-관리방식                     876125
k-복도유형                     876454
k-난방방식                     876125
k-전체동수                     877207
k-전체세대수                    876125
k-건설사(시공사)                 877637
k-시행사                      877834
k-사용검사일-사용승인일              876259
k-연면적                      876125
k-주거전용면적      

⏸ 역세권 피처 생성 시 필요한 칼럼들만 남김!

In [6]:
# 역세권 피처 생성 시 필요한 칼럼들만 남기기
important_columns = [
    '시군구', '번지', '도로명','좌표X', '좌표Y', '아파트명', 'is_test', 'target'
]

# 주요 칼럼만 남긴 데이터프레임
filtered_concat = concat[important_columns]

# 각 열의 결측치 수 확인
filtered_concat.isna().sum()

시군구             0
번지            227
도로명             0
좌표X        876232
좌표Y        876232
아파트명         2136
is_test         0
target       9272
dtype: int64

In [7]:
filtered_concat['구'] = filtered_concat['시군구'].map(lambda x : x.split()[1])
filtered_concat['동'] = filtered_concat['시군구'].map(lambda x : x.split()[2])

In [8]:
# '서초포레스타2단지' 아파트명의 '번지' 채우기
filtered_concat.loc[(filtered_concat['번지'].isna()) & (filtered_concat['아파트명'] == '서초포레스타2단지'), '번지'] = '384'

# '힐스테이트 서초 젠트리스' 아파트명의 '번지' 값을 557로 채워넣기
filtered_concat.loc[(filtered_concat['번지'].isna()) & (filtered_concat['아파트명'] == '힐스테이트 서초 젠트리스'), '번지'] = '557'

In [9]:
filtered_concat['아파트명'] = filtered_concat['아파트명'].fillna('NULL')

In [10]:
filtered_concat.isna().sum()

시군구             0
번지              0
도로명             0
좌표X        876232
좌표Y        876232
아파트명            0
is_test         0
target       9272
구               0
동               0
dtype: int64

## 1. X, Y 좌표 보완
Naver API를 통해 NA인 X, Y좌표를 보완 <br>
[🔗 Naver Geocoding API](https://api.ncloud-docs.com/docs/ai-naver-mapsgeocoding-geocode)

`?query={주소}`

In [11]:
df_xy_not_na = filtered_concat[(filtered_concat['좌표X'].notna()) & (filtered_concat['좌표Y'].notna())]

In [12]:
df_xy_na = filtered_concat[(filtered_concat['좌표X'].isna()) & (filtered_concat['좌표Y'].isna())]

In [13]:
df_xy_na.count()

시군구        876232
번지         876232
도로명        876232
좌표X             0
좌표Y             0
아파트명       876232
is_test    876232
target     869670
구          876232
동          876232
dtype: int64

In [14]:
df_xy_na.drop(columns=['target'], inplace=True)

In [15]:
df_xy_na.drop_duplicates(inplace=True)

In [16]:
df_xy_na.count()

시군구        10746
번지         10746
도로명        10746
좌표X            0
좌표Y            0
아파트명       10746
is_test    10746
구          10746
동          10746
dtype: int64

In [17]:
import requests

In [18]:
headers = {
    'X-NCP-APIGW-API-KEY-ID': NAVER_API_KEY_ID,
    'X-NCP-APIGW-API-KEY': NAVER_API_KEY
}

In [19]:
URL_GEO = 'https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode'

In [20]:
def error_log(file_path, data, response):
    with open(file_path, 'a') as f:
        f.write(f'{data}\n{response.status_code}\n{response.json()}\n---------\n')
    print(data, '\n[ERROR]', response.json())

좀 더 간결하게 짜고 싶은데 일단..................pass

In [21]:
def by_street_name(df, addr, i):
    addr = df.iloc[i]['도로명']
    query = f'?query={addr}'
    response = requests.get(URL_GEO + query, headers=headers)
    if response.status_code == 200:
        data = response.json()
        if (data['status'] == 'OK') & (data['meta']['totalCount'] > 0):
            x = data['addresses'][0]['x']
            y = data['addresses'][0]['y']
            df.iloc[i, df.columns.get_loc('좌표X')] = x
            df.iloc[i, df.columns.get_loc('좌표Y')] = y
        else:
            with open("주소확인.txt", 'a') as f:
                f.write(f'{addr}\n{data}\n---------\n')
    else:
        error_log("error_주소확인.txt", addr, response)

In [22]:
def fill_xy(df):
    try:
        for i in tqdm(range(len(df)), desc="Processing"):
            addr = f"{df.iloc[i]['시군구']} {df.iloc[i]['번지']}"
            query = f'?query={addr}'
            response = requests.get(URL_GEO + query, headers=headers)
            if response.status_code == 200:
                data = response.json()
                if (data['status'] == 'OK') & (data['meta']['totalCount'] > 0):
                    x = data['addresses'][0]['x']
                    y = data['addresses'][0]['y']
                    df.iloc[i, df.columns.get_loc('좌표X')] = x
                    df.iloc[i, df.columns.get_loc('좌표Y')] = y
                else:
                    # with open("주소확인.txt", 'a') as f:
                    #     f.write(f'{addr}\n{data}\n---------\n')
                    by_street_name(df, addr, i)
            else:
                error_log("error_주소확인.txt", addr, response)
    except Exception as e:
        error_log("error_주소확인.txt", addr, response)
        print(addr, '\n[ERROR]', e)

In [23]:
## 서버에서 실행 시 25분 정도 소요
fill_xy(df_xy_na)

Processing: 100%|██████████| 10746/10746 [25:00<00:00,  7.16it/s] 


In [24]:
df_xy_na.isna().sum()

시군구        0
번지         0
도로명        0
좌표X        1
좌표Y        1
아파트명       0
is_test    0
구          0
동          0
dtype: int64

도로명, 시군구+번지로도 검색 불가능한 row가 1개 존재

해당 row는 아파트명으로 검색한 실주소로 다시 API를 사용해 X좌표, Y좌표를 알아냄.

In [25]:
df_xy_na.loc[df_xy_na['좌표X'].isna()]

,시군구,번지,도로명,좌표X,좌표Y,아파트명,is_test,구,동
249335,서울특별시 관악구 봉천동,1152,27,NaN,NaN,관악푸르지오102동,0,관악구,봉천동


In [26]:
query = f'?query=서울시 관악구 봉천동 1717'
response = requests.get(URL_GEO + query, headers=headers)

In [27]:
df_xy_na.loc[df_xy_na['좌표X'].isna(), '좌표X'] = response.json()['addresses'][0]['x']
df_xy_na.loc[df_xy_na['좌표Y'].isna(), '좌표Y'] = response.json()['addresses'][0]['y']

In [28]:
df_xy_na.isna().sum()

시군구        0
번지         0
도로명        0
좌표X        0
좌표Y        0
아파트명       0
is_test    0
구          0
동          0
dtype: int64

In [29]:
df_xy_na.head()

,시군구,번지,도로명,좌표X,좌표Y,아파트명,is_test,구,동
975,서울특별시 강남구 개포동,189,삼성로 14,127.0713671,37.4855215,개포주공4단지,0,강남구,개포동
986,서울특별시 강남구 개포동,185,개포로 516,127.0716669,37.4888519,개포주공7단지,0,강남구,개포동
1031,서울특별시 강남구 개포동,655-1,언주로 105,127.0526829,37.4817013,개포현대200동,0,강남구,개포동
1034,서울특별시 강남구 개포동,649,언주로 110,127.0541788,37.4845189,경남1,0,강남구,개포동
1047,서울특별시 강남구 개포동,649,언주로 110,127.0541788,37.4845189,경남2차,0,강남구,개포동


In [30]:
df_empty_xy = filtered_concat[(filtered_concat['좌표X'].isna()) & (filtered_concat['좌표Y'].isna())]

In [31]:
df_empty_xy.drop(columns=['좌표X', '좌표Y'], inplace=True)

In [35]:
df_fill_xy = pd.merge(df_empty_xy, df_xy_na, on=['시군구', '구', '동', '번지', '도로명', '아파트명', 'is_test'], how='left')

In [36]:
df_fill_xy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 876232 entries, 0 to 876231
Data columns (total 10 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   시군구      876232 non-null  object 
 1   번지       876232 non-null  object 
 2   도로명      876232 non-null  object 
 3   아파트명     876232 non-null  object 
 4   is_test  876232 non-null  int64  
 5   target   869670 non-null  float64
 6   구        876232 non-null  object 
 7   동        876232 non-null  object 
 8   좌표X      876232 non-null  object 
 9   좌표Y      876232 non-null  object 
dtypes: float64(1), int64(1), object(8)
memory usage: 73.5+ MB


In [37]:
df_fill_xy.isna().sum()

시군구           0
번지            0
도로명           0
아파트명          0
is_test       0
target     6562
구             0
동             0
좌표X           0
좌표Y           0
dtype: int64

In [39]:
## 비어있던 좌표X, 좌표Y를 채운 데이터프레임과 기존에 이미 값이 있던 데이터프레임을 concat
df = pd.concat([df_xy_not_na, df_fill_xy])

In [40]:
df.isna().sum()

시군구           0
번지            0
도로명           0
좌표X           0
좌표Y           0
아파트명          0
is_test       0
target     9272
구             0
동             0
dtype: int64

In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1128094 entries, 0 to 876231
Data columns (total 10 columns):
 #   Column   Non-Null Count    Dtype  
---  ------   --------------    -----  
 0   시군구      1128094 non-null  object 
 1   번지       1128094 non-null  object 
 2   도로명      1128094 non-null  object 
 3   좌표X      1128094 non-null  object 
 4   좌표Y      1128094 non-null  object 
 5   아파트명     1128094 non-null  object 
 6   is_test  1128094 non-null  int64  
 7   target   1118822 non-null  float64
 8   구        1128094 non-null  object 
 9   동        1128094 non-null  object 
dtypes: float64(1), int64(1), object(8)
memory usage: 94.7+ MB


In [42]:
filtered_concat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1128094 entries, 0 to 9271
Data columns (total 10 columns):
 #   Column   Non-Null Count    Dtype  
---  ------   --------------    -----  
 0   시군구      1128094 non-null  object 
 1   번지       1128094 non-null  object 
 2   도로명      1128094 non-null  object 
 3   좌표X      251862 non-null   float64
 4   좌표Y      251862 non-null   float64
 5   아파트명     1128094 non-null  object 
 6   is_test  1128094 non-null  int64  
 7   target   1118822 non-null  float64
 8   구        1128094 non-null  object 
 9   동        1128094 non-null  object 
dtypes: float64(3), int64(1), object(6)
memory usage: 94.7+ MB


## 2. subway_feature 에 행정동 추가
이미 좌표X(경도), 좌표Y(위도)를 알고 있으므로, reverse geocoding을 통해 행정동을 알아낸다. <br>
[🔗 Naver Reverse Geocoding API](https://api.ncloud-docs.com/docs/ai-naver-mapsreversegeocoding-gc)

`?coords={입력_좌표}&sourcecrs={좌표계}&orders={변환_작업_이름}&output={출력_형식}`

<b>sourcecrs</b> 좌표계 <br>
default값(기본값)은 위경도 좌표계(epsg:4326)

<b>coords</b> 입력 좌표

<b>orders</b> 변환 작업 이름 <br>
`legalcode` 좌표 to 법정동 <br>
✅ `admcode` 좌표 to 행정동 <br>
`addr` 좌표 to 지번 주소

<b>output</b> 출력_형식<br>
사용 가능한 값은 json, xml<br>
default값(기본값)은 xml <br>
✅ `json`

In [43]:
ORDERS = 'admcode'
OUTPUT = 'json'

In [44]:
df_subway = pd.read_csv(PATH_SUBWAY)

In [45]:
df_subway.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   역사_ID   768 non-null    int64  
 1   역사명     768 non-null    object 
 2   호선      768 non-null    object 
 3   위도      768 non-null    float64
 4   경도      768 non-null    float64
dtypes: float64(2), int64(1), object(2)
memory usage: 30.1+ KB


In [46]:
URL_REVERSE_GEO = 'https://naveropenapi.apigw.ntruss.com/map-reversegeocode/v2/gc'

In [47]:
df_subway['시군구'] = np.nan
df_subway['시'] = np.nan
df_subway['구'] = np.nan
df_subway['동'] = np.nan

In [48]:
def fill_address_from_xy(df):
    try:
        for i in tqdm(range(len(df)), desc="Processing"):
            station = df.iloc[i]['역사명']
            x = df.iloc[i]['경도']
            y = df.iloc[i]['위도']
            query = f'?coords={x},{y}&orders={ORDERS}&output={OUTPUT}'
            response = requests.get(URL_REVERSE_GEO+query, headers=headers)
            if response.status_code == 200:
                data = response.json()
                if (data['status']['code'] == 0) & (len(data['results']) > 0):
                    address = f"{data['results'][0]['region']['area1']['name']} {data['results'][0]['region']['area2']['name']} {data['results'][0]['region']['area3']['name']}"
                    addr1 = data['results'][0]['region']['area1']['name']
                    addr2 = data['results'][0]['region']['area2']['name']
                    addr3 = data['results'][0]['region']['area3']['name']
                    df.iloc[i, df.columns.get_loc('시군구')] = address
                    df.iloc[i, df.columns.get_loc('시')] = addr1
                    df.iloc[i, df.columns.get_loc('구')] = addr2
                    df.iloc[i, df.columns.get_loc('동')] = addr3
                else:
                    with open("지하철주소확인.txt", 'a') as f:
                        f.write(f'{station}\n{data}\n---------\n')
            else:
                error_log("error_지하철주소확인.txt", station, response)
    except Exception as e:
        error_log("error_지하철주소확인.txt", station, response)
        print(station, '\n[ERROR]', e)

In [49]:
fill_address_from_xy(df_subway)

Processing: 100%|██████████| 768/768 [00:38<00:00, 20.13it/s]


In [50]:
df_subway

,역사_ID,역사명,호선,위도,경도,시군구,시,구,동
0,9996,미사,5호선,37.560927,127.193877,경기도 하남시 미사1동,경기도,하남시,미사1동
1,9995,강일,5호선,37.557490,127.175930,서울특별시 강동구 강일동,서울특별시,강동구,강일동
2,4929,김포공항,김포골드라인,37.562360,126.801868,서울특별시 강서구 방화2동,서울특별시,강서구,방화2동
3,4928,고촌,김포골드라인,37.601243,126.770345,경기도 김포시 고촌읍,경기도,김포시,고촌읍
4,4927,풍무,김포골드라인,37.612488,126.732387,경기도 김포시 사우동,경기도,김포시,사우동
...,...,...,...,...,...,...,...,...,...
763,154,종로5가,1호선,37.570926,127.001849,서울특별시 종로구 종로5.6가동,서울특별시,종로구,종로5.6가동
764,153,종로3가,1호선,37.570406,126.991847,서울특별시 종로구 종로1.2.3.4가동,서울특별시,종로구,종로1.2.3.4가동
765,152,종각,1호선,37.570161,126.982923,서울특별시 종로구 종로1.2.3.4가동,서울특별시,종로구,종로1.2.3.4가동
766,151,시청,1호선,37.565715,126.977088,서울특별시 중구 소공동,서울특별시,중구,소공동


In [51]:
df_subway.isna().sum()

역사_ID    0
역사명      0
호선       0
위도       0
경도       0
시군구      0
시        0
구        0
동        0
dtype: int64

## 3. 같은 '구'에 속한 아파트와 지하철역 거리 계산 후 역세권/비역세권 산정

- 소수점 2자리: 약 1.1킬로미터의 정확도
- 소수점 3자리: 약 110미터의 정확도
- <b> 소수점 4자리: 약 11미터의 정확도 </b>
- 소수점 5자리: 약 1.1미터의 정확도
- 소수점 6자리: 약 0.11미터의 정확도

평균 보행 속도로 계산 시 1분에 약 80미터

따라서 10미터를 걷는 데는 대략 10m / 80m/분 = 0.125분, 즉 약 7.5초

계산의 용이성을 위해 좌표X, 좌표Y의 단위를 소수점 4자리로 round 하여 사용함

어차피 '구'단위로 비교해서 계산을 한다면 df_subway에서 서울특별시가 아닌 row들은 제거해도 괜찮았을 듯 (크게 의미 x)

In [52]:
df['좌표Y'] = df['좌표Y'].astype(float).round(4)
df['좌표X'] = df['좌표X'].astype(float).round(4)
df_subway['위도'] = df_subway['위도'].round(4)
df_subway['경도'] = df_subway['경도'].round(4)

In [53]:
# 시군구, 년월 등 분할할 수 있는 변수들은 세부사항 고려를 용이하게 하기 위해 모두 분할해 주겠습니다.
df['구'] = df['시군구'].map(lambda x : x.split()[1])
df['동'] = df['시군구'].map(lambda x : x.split()[2])

In [80]:
df.columns

Index(['시군구', '번지', '도로명', '좌표X', '좌표Y', '아파트명', 'is_test', 'target', '구',
       '동'],
      dtype='object')

In [81]:
important_columns = ['시군구', '번지', '도로명', '구', '동', '아파트명', '좌표X', '좌표Y']

df_compare = df[important_columns]

In [82]:
df_compare.drop_duplicates(inplace=True)

In [83]:
df_compare.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9519 entries, 0 to 876024
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   시군구     9519 non-null   object 
 1   번지      9519 non-null   object 
 2   도로명     9519 non-null   object 
 3   구       9519 non-null   object 
 4   동       9519 non-null   object 
 5   아파트명    9519 non-null   object 
 6   좌표X     9519 non-null   float64
 7   좌표Y     9519 non-null   float64
dtypes: float64(2), object(6)
memory usage: 669.3+ KB


In [84]:
from geopy.distance import geodesic

In [85]:
def haversine(point1, point2):
    distance = geodesic(point1, point2).meters    
    is_in_500m = distance <= 500
    return is_in_500m


In [86]:
df_compare['역세권'] = np.nan

#### [회고 🦧]
발단: baseline_code에서 좌표X, 좌표Y가 선형 보간 처리가 된 것을 파악하지 못 함

역세권 관련해 역 개수가 유의미한 듯 하여 500m 이내의 역 개수를 피처로 추가했는데 성능 하락 --> <br>
인근 지하철 역 수 자체보다는, 지역구가 target에 더 큰 영향을 미치는 것이 원인으로 판단 --> <br>
"지역구 + 역세권 여부" 형태의 범주형 데이터를 추가하는 것으로 방향성을 잡음

이후 선형 보간 처리가 된 것을 확인하고 주소로 좌표X, 좌표Y를 API를 통해 보완했으나 방향성은 바꾸지 않았는데

겹치는 역 개수, 지하철 노선도 중요도 등을 적용했으면 좀 더 유의미한 피처 엔지니어링이 되었을 것 같다

In [87]:
def compare_distance(df, subway):
    ## 구 단위로 비교
    try:
        for i in tqdm(range(len(df)), desc="Processing"):
            area = df.iloc[i]['구']
            filtered = subway[subway['구'] == area]
            point1 = (df.iloc[i]['좌표Y'], df.iloc[i]['좌표X'])
            cnt = 0
            for j in range(len(filtered)):
                point2 = (filtered.iloc[j]['위도'], filtered.iloc[j]['경도'])
                is_in_500m = haversine(point1, point2)
                if is_in_500m:
                    cnt+=1
                if cnt:
                    is_close = area + ' 역세권'
                else:
                    is_close = area + ' 비역세권'
                df.iloc[i, df.columns.get_loc('역세권')] = is_close
            # print(df.iloc[i]['아파트명'], point1, "의 인근역 수:", cnt, ",", is_close)
    except Exception as e:
        print('[ERROR]', df.iloc[i], e)

In [88]:
compare_distance(df_compare, df_subway)

Processing: 100%|██████████| 9519/9519 [00:51<00:00, 186.27it/s]


In [89]:
df_compare.head()

,시군구,번지,도로명,구,동,아파트명,좌표X,좌표Y,역세권
0,서울특별시 강남구 개포동,658-1,언주로 3,강남구,개포동,개포6차우성,127.0572,37.4768,강남구 비역세권
12,서울특별시 강남구 개포동,652,개포로 307,강남구,개포동,개포우성3차,127.0560,37.4839,강남구 역세권
25,서울특별시 강남구 개포동,12-2,개포로109길 69,강남구,개포동,개포자이,127.0766,37.4963,강남구 역세권
38,서울특별시 강남구 개포동,141,개포로 310,강남구,개포동,개포주공1단지,127.0585,37.4800,강남구 비역세권
44,서울특별시 강남구 개포동,141,선릉로 7,강남구,개포동,개포주공1단지,127.0585,37.4800,강남구 비역세권


In [90]:
df_compare.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9519 entries, 0 to 876024
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   시군구     9519 non-null   object 
 1   번지      9519 non-null   object 
 2   도로명     9519 non-null   object 
 3   구       9519 non-null   object 
 4   동       9519 non-null   object 
 5   아파트명    9519 non-null   object 
 6   좌표X     9519 non-null   float64
 7   좌표Y     9519 non-null   float64
 8   역세권     9519 non-null   object 
dtypes: float64(2), object(7)
memory usage: 743.7+ KB


In [91]:
df_compare.isna().sum()

시군구     0
번지      0
도로명     0
구       0
동       0
아파트명    0
좌표X     0
좌표Y     0
역세권     0
dtype: int64

## 4. 각 구 역세권/비역세권별 target 평균으로 랭크 산정
범주형 칼럼으로 모델이 학습을 잘 할 수 있을지 우려 + 이후 다중공산성을 완화하고자 역세권에서 구명을 제거하기 때문에, <br>
target 평균으로 랭크를 산정하여 피처로 추가함.

시기적인 고려 없이 아주 단순한 계산으로 '역세권'별 전체 target의 mean으로 계산했기 때문에, 좀 더 개선의 필요성이 있음. 

In [92]:
filtered_concat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1128094 entries, 0 to 9271
Data columns (total 8 columns):
 #   Column   Non-Null Count    Dtype  
---  ------   --------------    -----  
 0   시군구      1128094 non-null  object 
 1   번지       1128094 non-null  object 
 2   도로명      1128094 non-null  object 
 3   아파트명     1128094 non-null  object 
 4   is_test  1128094 non-null  int64  
 5   target   1118822 non-null  float64
 6   구        1128094 non-null  object 
 7   동        1128094 non-null  object 
dtypes: float64(1), int64(1), object(6)
memory usage: 77.5+ MB


In [ ]:
del filtered_concat['좌표X']
del filtered_concat['좌표Y']

In [98]:
df_merged = pd.merge(filtered_concat, df_compare, on=['시군구', '번지', '도로명', '아파트명', '구', '동'], how='left')

In [99]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1128094 entries, 0 to 1128093
Data columns (total 11 columns):
 #   Column   Non-Null Count    Dtype  
---  ------   --------------    -----  
 0   시군구      1128094 non-null  object 
 1   번지       1128094 non-null  object 
 2   도로명      1128094 non-null  object 
 3   아파트명     1128094 non-null  object 
 4   is_test  1128094 non-null  int64  
 5   target   1118822 non-null  float64
 6   구        1128094 non-null  object 
 7   동        1128094 non-null  object 
 8   좌표X      1128094 non-null  float64
 9   좌표Y      1128094 non-null  float64
 10  역세권      1128094 non-null  object 
dtypes: float64(3), int64(1), object(7)
memory usage: 103.3+ MB


In [100]:
filtered_concat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1128094 entries, 0 to 9271
Data columns (total 8 columns):
 #   Column   Non-Null Count    Dtype  
---  ------   --------------    -----  
 0   시군구      1128094 non-null  object 
 1   번지       1128094 non-null  object 
 2   도로명      1128094 non-null  object 
 3   아파트명     1128094 non-null  object 
 4   is_test  1128094 non-null  int64  
 5   target   1118822 non-null  float64
 6   구        1128094 non-null  object 
 7   동        1128094 non-null  object 
dtypes: float64(1), int64(1), object(6)
memory usage: 77.5+ MB


In [103]:
filtered_group = df_merged.groupby(['구', '역세권'])['target'].apply(lambda x: list(x.unique())).reset_index()

In [105]:
filtered_group

,구,역세권,target
0,강남구,강남구 비역세권,"[124000.0, 123500.0, 91500.0, 130000.0, 117000..."
1,강남구,강남구 역세권,"[152000.0, 182000.0, 170000.0, 185000.0, 17500..."
2,강동구,강동구 비역세권,"[54000.0, 55500.0, 53900.0, 57000.0, 56000.0, ..."
3,강동구,강동구 역세권,"[59500.0, 60000.0, 53800.0, 58000.0, 59000.0, ..."
4,강북구,강북구 비역세권,"[45000.0, 65500.0, 58900.0, 60000.0, 56300.0, ..."
5,강북구,강북구 역세권,"[42500.0, 39000.0, 44200.0, 38600.0, 45200.0, ..."
6,강서구,강서구 비역세권,"[32750.0, 30800.0, 25000.0, 25900.0, 39300.0, ..."
7,강서구,강서구 역세권,"[32300.0, 33900.0, 46500.0, 32000.0, 42300.0, ..."
8,관악구,관악구 비역세권,"[55500.0, 53000.0, 71500.0, 56000.0, 65500.0, ..."
9,관악구,관악구 역세권,"[40800.0, 56000.0, 35400.0, 29800.0, 31200.0, ..."


In [106]:
## 배열의 평균을 계산하여 새로운 컬럼 추가
filtered_group['평균_target'] = filtered_group['target'].apply(lambda x: pd.Series(x).mean())

## 평균값을 기준으로 순위 매기기
filtered_group['구_역세권_랭크'] = filtered_group['평균_target'].rank(ascending=False)

In [112]:
filtered_group.sort_values(by='구_역세권_랭크')

,구,역세권,target,평균_target,구_역세권_랭크
40,용산구,용산구 비역세권,"[62500.0, 82000.0, 67000.0, 62800.0, 54000.0, ...",152757.601347,1.0
29,서초구,서초구 역세권,"[343000.0, 195000.0, 200000.0, 323000.0, 19700...",141118.294639,2.0
1,강남구,강남구 역세권,"[152000.0, 182000.0, 170000.0, 185000.0, 17500...",133830.482510,3.0
0,강남구,강남구 비역세권,"[124000.0, 123500.0, 91500.0, 130000.0, 117000...",126924.633595,4.0
41,용산구,용산구 역세권,"[81700.0, 79800.0, 80000.0, 81800.0, 82000.0, ...",125740.910726,5.0
28,서초구,서초구 비역세권,"[96000.0, 67964.0, 106000.0, 114000.0, 71910.0...",104453.256103,6.0
34,송파구,송파구 비역세권,"[54500.0, 54800.0, 55500.0, 52000.0, 57250.0, ...",98854.395026,7.0
31,성동구,성동구 역세권,"[63000.0, 63600.0, 61800.0, 62300.0, 75000.0, ...",98335.438865,8.0
35,송파구,송파구 역세권,"[52000.0, 59500.0, 57500.0, 69500.0, 70000.0, ...",93288.954079,9.0
30,성동구,성동구 비역세권,"[59000.0, 58000.0, 70000.0, 74800.0, 59700.0, ...",89457.806867,10.0


In [113]:
filtered_group.isna().sum()

구            0
역세권          0
target       0
평균_target    0
구_역세권_랭크     0
dtype: int64

In [114]:
len(filtered_group['구_역세권_랭크'].unique())

50

In [115]:
df_merged.columns

Index(['시군구', '번지', '도로명', '아파트명', 'is_test', 'target', '구', '동', '좌표X', '좌표Y',
       '역세권'],
      dtype='object')

In [116]:
filtered_group.columns

Index(['구', '역세권', 'target', '평균_target', '구_역세권_랭크'], dtype='object')

In [117]:
del filtered_group['target']
del filtered_group['평균_target']

In [174]:
df_merged_rank = pd.merge(df_merged, filtered_group, on=['구', '역세권'], how='left')

In [175]:
df_merged_rank.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1128094 entries, 0 to 1128093
Data columns (total 12 columns):
 #   Column    Non-Null Count    Dtype  
---  ------    --------------    -----  
 0   시군구       1128094 non-null  object 
 1   번지        1128094 non-null  object 
 2   도로명       1128094 non-null  object 
 3   아파트명      1128094 non-null  object 
 4   is_test   1128094 non-null  int64  
 5   target    1118822 non-null  float64
 6   구         1128094 non-null  object 
 7   동         1128094 non-null  object 
 8   좌표X       1128094 non-null  float64
 9   좌표Y       1128094 non-null  float64
 10  역세권       1128094 non-null  object 
 11  구_역세권_랭크  1128094 non-null  float64
dtypes: float64(4), int64(1), object(7)
memory usage: 111.9+ MB


In [176]:
df_merged_rank.isna().sum()

시군구            0
번지             0
도로명            0
아파트명           0
is_test        0
target      9272
구              0
동              0
좌표X            0
좌표Y            0
역세권            0
구_역세권_랭크       0
dtype: int64

In [177]:
## '역세권'에 '구'의 데이터가 포함되어 있어 다중공산성이 매우 높은 문제를 해결하기 위해 행정구명 제거
df_merged_rank['역세권'] = df_merged_rank['역세권'].apply(lambda x: x.split(' ')[1])

In [179]:
df_merged_rank.loc[df_merged_rank['target'].isna(), 'target'] = 0

In [180]:
df_merged_rank['주소'] = df_merged_rank['동'] + ' ' + df_merged_rank['번지']

In [181]:
df_merged_rank.isna().sum()

시군구         0
번지          0
도로명         0
아파트명        0
is_test     0
target      0
구           0
동           0
좌표X         0
좌표Y         0
역세권         0
구_역세권_랭크    0
주소          0
dtype: int64

In [165]:
## 위에서 제작해 놓았던 is_test 칼럼을 이용해 데이터셋 수에 변화가 생기진 않았는지 점검
train_set = df_merged_rank.query('is_test==0')
test_set = df_merged_rank.query('is_test==1')
print(train_set.shape, test_set.shape)

(1118822, 13) (9272, 13)


⏸ '이전_N번째_거래가격'이 추가된 데이터프레임을 불러와 마저 작업

In [142]:
## csv 읽을 때 타입 변환 방지
dict_dtype = {
    "본번" : str,
    "부번" : str,
    "아파트명" : str,
    "계약년" : str,
    "계약월" : str,
    "계약일" : str
}

In [143]:
df_final = pd.read_csv(PATH_RECENT, keep_default_na=False, dtype=dict_dtype)

In [144]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1128094 entries, 0 to 1128093
Data columns (total 29 columns):
 #   Column        Non-Null Count    Dtype  
---  ------        --------------    -----  
 0   번지            1128094 non-null  object 
 1   본번            1128094 non-null  object 
 2   부번            1128094 non-null  object 
 3   아파트명          1128094 non-null  object 
 4   전용면적(㎡)       1128094 non-null  float64
 5   계약년월          1128094 non-null  int64  
 6   계약일           1128094 non-null  object 
 7   층             1128094 non-null  int64  
 8   건축년도          1128094 non-null  int64  
 9   도로명           1128094 non-null  object 
 10  거래유형          1128094 non-null  object 
 11  중개사소재지        1128094 non-null  object 
 12  target        1128094 non-null  float64
 13  is_test       1128094 non-null  int64  
 14  구             1128094 non-null  object 
 15  동             1128094 non-null  object 
 16  계약년           1128094 non-null  object 
 17  계약월           1128094 non-n

In [183]:
df_merged_rank = df_merged_rank[['is_test', '주소', '역세권', '구_역세권_랭크']]

In [184]:
overlapped = list(set(df_final.columns) & set(df_merged_rank.columns))
overlapped

['주소', 'is_test']

In [186]:
df_merged_rank.drop_duplicates(inplace=True)

In [187]:
df_merged_rank.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11531 entries, 0 to 1128089
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   is_test   11531 non-null  int64  
 1   주소        11531 non-null  object 
 2   역세권       11531 non-null  object 
 3   구_역세권_랭크  11531 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 450.4+ KB


In [188]:
df_final_merged = pd.merge(df_final, df_merged_rank, on=overlapped, how='left')

In [189]:
## 위에서 제작해 놓았던 is_test 칼럼을 이용해 데이터셋 수에 변화가 생기진 않았는지 점검
train_set = df_final_merged.query('is_test==0')
test_set = df_final_merged.query('is_test==1')
print(train_set.shape, test_set.shape)

(1118822, 31) (9272, 31)


## 모델링

In [200]:
# Model
from catboost import CatBoostRegressor, Pool

from sklearn.model_selection import KFold

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [193]:
columns_to_drop = ['계약년', '번지', '주소']

In [194]:
df_final_merged.drop(columns=columns_to_drop, inplace=True)

In [201]:
# 이제 다시 train과 test dataset을 분할해줍니다. 위에서 제작해 놓았던 is_test 칼럼을 이용합니다.
dt_train = df_final_merged.query('is_test==0')
dt_test = df_final_merged.query('is_test==1')

# 이제 is_test 칼럼은 drop해줍니다.
dt_train.drop(['is_test'], axis = 1, inplace=True)
dt_test.drop(['is_test'], axis = 1, inplace=True)
print(dt_train.shape, dt_test.shape)

(1118822, 27) (9272, 27)


In [202]:
# 파생변수 제작으로 추가된 변수들이 존재하기에, 다시한번 연속형과 범주형 칼럼을 분리해주겠습니다.
continuous_columns_v2 = []
categorical_columns_v2 = []

for column in dt_train.columns:
    if pd.api.types.is_numeric_dtype(dt_train[column]):
        continuous_columns_v2.append(column)
    else:
        categorical_columns_v2.append(column)

print("연속형 변수:", continuous_columns_v2)
print("범주형 변수:", categorical_columns_v2)

연속형 변수: ['전용면적(㎡)', '계약년월', '층', '건축년도', 'target', '이전_1번째_거래가격', '이전_2번째_거래가격', '이전_3번째_거래가격', '이전_4번째_거래가격', '이전_5번째_거래가격', '이전_6번째_거래가격', '이전_7번째_거래가격', '이전_8번째_거래가격', '이전_9번째_거래가격', '이전_10번째_거래가격', '구_역세권_랭크']
범주형 변수: ['본번', '부번', '아파트명', '계약일', '도로명', '거래유형', '중개사소재지', '구', '동', '계약월', '역세권']


In [203]:
# 아래에서 범주형 변수들을 대상으로 레이블인코딩을 진행해 주겠습니다.

# 각 변수에 대한 LabelEncoder를 저장할 딕셔너리
label_encoders = {}

# Implement Label Encoding
for col in tqdm( categorical_columns_v2 ):
    lbl = LabelEncoder()

    # Label-Encoding을 fit
    lbl.fit( dt_train[col].astype(str) )
    dt_train[col] = lbl.transform(dt_train[col].astype(str))
    label_encoders[col] = lbl           # 나중에 후처리를 위해 레이블인코더를 저장해주겠습니다.

    # Test 데이터에만 존재하는 새로 출현한 데이터를 신규 클래스로 추가해줍니다.
    for label in np.unique(dt_test[col]):
      if label not in lbl.classes_: # unseen label 데이터인 경우
        lbl.classes_ = np.append(lbl.classes_, label) # 미처리 시 ValueError발생하니 주의하세요!

    dt_test[col] = lbl.transform(dt_test[col].astype(str))

100%|██████████| 11/11 [00:02<00:00,  4.24it/s]


In [204]:
assert dt_train.shape[1] == dt_test.shape[1]          # train/test dataset의 shape이 같은지 확인해주겠습니다.

In [206]:
# Target과 독립변수들을 분리해줍니다.
y_train = dt_train['target']
X_train = dt_train.drop(['target'], axis=1)

In [207]:
# K-fold 교차 검증을 위한 K 값 설정
k = 5  # 예시로 K=5로 설정
# KFold 객체 생성
kf = KFold(n_splits=k, shuffle=True, random_state=2023)
# 모델 성능을 저장할 리스트
train_rmse_scores = []
val_rmse_scores = []
models = []  # 각 fold에서 선택된 모델을 저장할 리스트
params = {
    'iterations': 2500,
    'learning_rate': 0.2619829692634429,
    'depth': 10,
    'l2_leaf_reg': 0.06339510160096859,
    'bootstrap_type': 'MVS',
    'subsample': 0.6742258586023032,
    'random_strength': 6.088205676048116,
    'min_data_in_leaf': 16,
    'loss_function': 'RMSE',
    'eval_metric': 'RMSE'
}
# K-fold 반복
for train_index, val_index in kf.split(X_train):
    # K-fold 내의 학습 데이터와 검증 데이터
    X_train_kf, X_val_kf = X_train.iloc[train_index], X_train.iloc[val_index]
    y_train_kf, y_val_kf = y_train.iloc[train_index], y_train.iloc[val_index]
    # CatBoostRegressor를 이용해 회귀 모델을 학습
    model = CatBoostRegressor(**params)
    model.fit(X_train_kf, y_train_kf, eval_set=(X_val_kf, y_val_kf), early_stopping_rounds=50, verbose=100, plot=True)
    # 검증 데이터에서 예측값 계산
    val_pred = model.predict(X_val_kf)
    # 회귀 모델의 성능 지표인 RMSE(Root Mean Squared Error) 계산
    val_rmse = np.sqrt(mean_squared_error(y_val_kf, val_pred))
    train_rmse = np.sqrt(mean_squared_error(y_train_kf, model.predict(X_train_kf)))
    # 학습 결과 추출
    evals_result = model.get_evals_result()
    # 학습 및 검증 데이터의 RMSE 값 추출
    train_rmse_curve = evals_result['learn']['RMSE']
    val_rmse_curve = evals_result['validation']['RMSE']
    # # 학습 곡선 시각화
    # plt.figure(figsize=(10, 6))
    # plt.plot(train_rmse_curve, label='Train RMSE')
    # plt.plot(val_rmse_curve, label='Validation RMSE')
    # plt.xlim(100, 2500)
    # plt.ylim(3000, 7000)
    # plt.xlabel('Iteration')
    # plt.ylabel('RMSE')
    # plt.legend()
    # plt.title('Learning and Validation Curves')
    # plt.show()
    # 각 fold에서의 성능 점수 저장
    train_rmse_scores.append(train_rmse)
    val_rmse_scores.append(val_rmse)
    models.append(model)  # 해당 모델을 리스트에 저장
# 가장 성능이 좋은(즉, val_rmse가 최소인) 모델 선택
best_model_index = np.argmin(val_rmse_scores)
best_model = models[best_model_index]
# K-fold 교차 검증 결과 출력
print(f'K-fold 교차 검증 {k}번 min RMSE:')
print(f'- 학습 데이터 min RMSE: {np.min(train_rmse_scores):.4f}')
print(f'- 검증 데이터 min RMSE: {np.min(val_rmse_scores):.4f}')

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 36846.3454741	test: 37196.4468266	best: 37196.4468266 (0)	total: 113ms	remaining: 4m 42s
100:	learn: 6643.3474380	test: 7301.1780529	best: 7301.1780529 (100)	total: 6.47s	remaining: 2m 33s
200:	learn: 5488.7405199	test: 6461.2582836	best: 6461.2582836 (200)	total: 12.8s	remaining: 2m 26s
300:	learn: 4916.8466866	test: 6128.9131549	best: 6128.9131549 (300)	total: 19s	remaining: 2m 18s
400:	learn: 4540.9798476	test: 5968.7504392	best: 5968.7504392 (400)	total: 24.8s	remaining: 2m 10s
500:	learn: 4276.1137494	test: 5867.4562542	best: 5867.4562542 (500)	total: 30.7s	remaining: 2m 2s
600:	learn: 4064.4241810	test: 5799.6360894	best: 5799.6360894 (600)	total: 36.6s	remaining: 1m 55s
700:	learn: 3894.7855580	test: 5750.7147384	best: 5750.7147384 (700)	total: 42.4s	remaining: 1m 48s
800:	learn: 3745.2413172	test: 5722.5918732	best: 5722.5918732 (800)	total: 48.3s	remaining: 1m 42s
900:	learn: 3627.0840276	test: 5700.3914471	best: 5699.1546755 (893)	total: 54.4s	remaining: 1m 36s
1000

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 36991.6792304	test: 36708.9299846	best: 36708.9299846 (0)	total: 63.4ms	remaining: 2m 38s
100:	learn: 6757.9009083	test: 7208.5829464	best: 7208.5829464 (100)	total: 5.96s	remaining: 2m 21s
200:	learn: 5531.1911405	test: 6379.6084530	best: 6379.6084530 (200)	total: 11.8s	remaining: 2m 14s
300:	learn: 4963.3715565	test: 6066.7274272	best: 6066.7274272 (300)	total: 17.7s	remaining: 2m 9s
400:	learn: 4586.2655890	test: 5894.6487553	best: 5894.6487553 (400)	total: 23.6s	remaining: 2m 3s
500:	learn: 4307.6221648	test: 5803.2105808	best: 5801.2657868 (494)	total: 29.5s	remaining: 1m 57s
600:	learn: 4091.7404927	test: 5750.1807923	best: 5749.3324097 (593)	total: 35.4s	remaining: 1m 51s
700:	learn: 3923.9748188	test: 5715.1800915	best: 5713.1049549 (699)	total: 41.2s	remaining: 1m 45s
800:	learn: 3778.5765915	test: 5675.3043015	best: 5675.3043015 (800)	total: 47.1s	remaining: 1m 39s
900:	learn: 3656.7628944	test: 5647.6471909	best: 5647.6471909 (900)	total: 53s	remaining: 1m 34s
1000

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 37054.1888874	test: 36944.6235506	best: 36944.6235506 (0)	total: 60.7ms	remaining: 2m 31s
100:	learn: 6677.6846481	test: 7072.3645667	best: 7072.3645667 (100)	total: 5.95s	remaining: 2m 21s
200:	learn: 5494.0030107	test: 6187.0518787	best: 6187.0518787 (200)	total: 11.7s	remaining: 2m 14s
300:	learn: 4911.0398131	test: 5849.0422084	best: 5848.4041530 (299)	total: 17.6s	remaining: 2m 8s
400:	learn: 4534.0971154	test: 5653.0134908	best: 5653.0134908 (400)	total: 23.5s	remaining: 2m 2s
500:	learn: 4266.8253925	test: 5548.1798556	best: 5548.1798556 (500)	total: 29.4s	remaining: 1m 57s
600:	learn: 4057.5007203	test: 5475.5160024	best: 5475.5160024 (600)	total: 35.3s	remaining: 1m 51s
700:	learn: 3889.6945721	test: 5421.2401088	best: 5421.2401088 (700)	total: 41.2s	remaining: 1m 45s
800:	learn: 3750.3766067	test: 5394.5034252	best: 5393.8796575 (799)	total: 47.1s	remaining: 1m 39s
900:	learn: 3629.7493301	test: 5362.8508615	best: 5362.8508615 (900)	total: 52.9s	remaining: 1m 33s
10

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 36869.9958896	test: 37098.5219662	best: 37098.5219662 (0)	total: 62.3ms	remaining: 2m 35s
100:	learn: 6651.4758757	test: 7219.5358625	best: 7219.5358625 (100)	total: 5.94s	remaining: 2m 21s
200:	learn: 5495.5766292	test: 6383.7810834	best: 6383.7810834 (200)	total: 11.8s	remaining: 2m 15s
300:	learn: 4912.7864197	test: 6043.2062409	best: 6043.2062409 (300)	total: 17.8s	remaining: 2m 9s
400:	learn: 4544.2650317	test: 5895.3674168	best: 5894.9015523 (399)	total: 23.7s	remaining: 2m 3s
500:	learn: 4269.6832024	test: 5780.6695932	best: 5780.6695932 (500)	total: 29.6s	remaining: 1m 58s
600:	learn: 4052.5937491	test: 5724.8876195	best: 5724.8876195 (600)	total: 35.5s	remaining: 1m 52s
700:	learn: 3880.7646537	test: 5666.4926781	best: 5666.4926781 (700)	total: 41.6s	remaining: 1m 46s
800:	learn: 3739.8790493	test: 5638.3000810	best: 5638.3000810 (800)	total: 47.6s	remaining: 1m 40s
900:	learn: 3620.4232902	test: 5614.4826937	best: 5614.4826937 (900)	total: 53.5s	remaining: 1m 34s
10

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 37143.4389527	test: 36957.2890797	best: 36957.2890797 (0)	total: 59.3ms	remaining: 2m 28s
100:	learn: 6840.5768629	test: 7205.3049306	best: 7205.3049306 (100)	total: 5.97s	remaining: 2m 21s
200:	learn: 5602.2677453	test: 6319.6625205	best: 6319.6625205 (200)	total: 11.9s	remaining: 2m 15s
300:	learn: 4985.8952364	test: 5995.3867350	best: 5995.3867350 (300)	total: 17.7s	remaining: 2m 9s
400:	learn: 4608.3116784	test: 5828.7285676	best: 5828.7285676 (400)	total: 23.6s	remaining: 2m 3s
500:	learn: 4325.7029615	test: 5726.8658201	best: 5726.8658201 (500)	total: 29.6s	remaining: 1m 57s
600:	learn: 4104.0210979	test: 5670.7548917	best: 5670.7548917 (600)	total: 35.5s	remaining: 1m 52s
700:	learn: 3932.9189172	test: 5632.7234864	best: 5632.1493428 (698)	total: 41.5s	remaining: 1m 46s
800:	learn: 3794.5346485	test: 5607.8209479	best: 5607.8209479 (800)	total: 47.4s	remaining: 1m 40s
900:	learn: 3662.3720824	test: 5592.7833703	best: 5592.7833703 (900)	total: 53.3s	remaining: 1m 34s
10

In [208]:
%%time
X_test = dt_test.drop(['target'], axis=1)

# Test dataset에 대한 inference를 진행합니다.
real_test_pred = model.predict(X_test)

CPU times: user 58.3 ms, sys: 159 µs, total: 58.4 ms
Wall time: 9.37 ms


In [209]:
# 앞서 예측한 예측값들을 저장합니다.
preds_df = pd.DataFrame(real_test_pred.astype(int), columns=["target"])
preds_df.to_csv('final_output_result.csv', index=False)